In [30]:
#write function that takes query input and returns recommendation list

In [537]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import tree
import random

In [464]:
import collections

In [527]:
#load data
cols=pd.read_csv("columns.csv")
resp=pd.read_csv("responses.csv")
podcasts=pd.read_csv("df_popular_podcasts.csv")
movies=pd.read_csv("MovieGenre.csv")
music=pd.read_csv("SpotifyFeatures.csv")
genre_IDs=[['1311', 'News & Politics'], ['26', 'Podcasts'], ['1479', 'Social Sciences'], ['1315', 'Science & Medicine'], ['1324', 'Society & Culture'], ['1302', 'Personal Journals'], ['1469', 'Language Courses'], ['1304', 'Education'], ['1320', 'Places & Travel'], ['1416', 'Higher Education'], ['1465', 'Professional'], ['1316', 'Sports & Recreation'], ['1303', 'Comedy'], ['1305', 'Kids & Family'], ['1439', 'Christianity'], ['1314', 'Religion & Spirituality'], ['1444', 'Spirituality'], ['1309', 'TV & Film'], ['1462', 'History'], ['1310', 'Music'], ['1478', 'Medicine'], ['1321', 'Business'], ['1412', 'Investing'], ['1420', 'Self-Help'], ['1307', 'Health'], ['1481', 'Alternative Health'], ['1417', 'Fitness & Nutrition'], ['1467', 'Amateur'], ['1480', 'Software How-To'], ['1318', 'Technology'], ['1448', 'Tech News'], ['1456', 'Outdoor'], ['1477', 'Natural Sciences'], ['1301', 'Arts'], ['1454', 'Automotive'], ['1323', 'Games & Hobbies'], ['1438', 'Buddhism'], ['1443', 'Philosophy'], ['1401', 'Literature'], ['1402', 'Design'], ['1410', 'Careers'], ['1470', 'Training'], ['1413', 'Management & Marketing'], ['1306', 'Food'], ['1406', 'Visual Arts'], ['1446', 'Gadgets'], ['1468', 'Educational Technology'], ['1405', 'Performing Arts'], ['1460', 'Hobbies'], ['1471', 'Business News'], ['1404', 'Video Games'], ['1450', 'Podcasting'], ['1473', 'National'], ['1325', 'Government & Organizations'], ['1461', 'Other Games'], ['1466', 'College & High School'], ['1459', 'Fashion & Beauty'], ['1476', 'Non-Profit'], ['1415', 'K-12'], ['1455', 'Aviation'], ['1464', 'Other'], ['1421', 'Sexuality'], ['1472', 'Shopping'], ['1475', 'Local'], ['1441', 'Judaism'], ['1440', 'Islam'], ['1474', 'Regional'], ['1463', 'Hinduism']]

In [522]:
movies=movies[20000:]

In [523]:
genre_to_music={}

for i in range(len(music)):
    genre=music['genre'][i]
    if genre in genre_to_music:
            genre_to_music[genre].append(music['track_name'][i]+" by "+music['artist_name'][i])
    else:
            genre_to_music[genre]=[music['track_name'][i]+ " by "+music['artist_name'][i]]

In [534]:
# Better organize the id vs name of genre
dict_pod={int(id[0]):id[1] for id in genre_IDs }

# {Genre ID:NAMES OF PODCAST} 
genre_to_podcast={key:[] for key in dict_pod }

for item in podcasts.index:
    array=podcasts["Genre IDs"][item][1:-1].split(',')
    array=[int(array[i].replace('\'', '').strip('\'')) for i in range(len(array))]
    for j in range(len(array)):
        genre_to_podcast[array[j]].append(podcasts["Name"][item])

final_dict={list(dict_pod.values())[p]:list(genre_to_podcast.values())[p] for p in range(len(genre_to_podcast.keys()))}

In [37]:
print(final_dict.keys())

dict_keys(['News & Politics', 'Podcasts', 'Social Sciences', 'Science & Medicine', 'Society & Culture', 'Personal Journals', 'Language Courses', 'Education', 'Places & Travel', 'Higher Education', 'Professional', 'Sports & Recreation', 'Comedy', 'Kids & Family', 'Christianity', 'Religion & Spirituality', 'Spirituality', 'TV & Film', 'History', 'Music', 'Medicine', 'Business', 'Investing', 'Self-Help', 'Health', 'Alternative Health', 'Fitness & Nutrition', 'Amateur', 'Software How-To', 'Technology', 'Tech News', 'Outdoor', 'Natural Sciences', 'Arts', 'Automotive', 'Games & Hobbies', 'Buddhism', 'Philosophy', 'Literature', 'Design', 'Careers', 'Training', 'Management & Marketing', 'Food', 'Visual Arts', 'Gadgets', 'Educational Technology', 'Performing Arts', 'Hobbies', 'Business News', 'Video Games', 'Podcasting', 'National', 'Government & Organizations', 'Other Games', 'College & High School', 'Fashion & Beauty', 'Non-Profit', 'K-12', 'Aviation', 'Other', 'Sexuality', 'Shopping', 'Local

In [528]:
genre_to_movie={}
for i in range(len(movies)):
    line=str(movies["Genre"][i])
    for genre in line.split("|"):
        genre=genre.lower()
        if genre in genre_to_movie:
            genre_to_movie[genre].append(movies["Title"][i])
        else: 
            genre_to_movie[genre]=[movies["Title"][i]]

In [529]:
print(genre_to_movie.keys())

dict_keys(['animation', 'adventure', 'comedy', 'action', 'family', 'romance', 'drama', 'crime', 'thriller', 'fantasy', 'horror', 'biography', 'history', 'mystery', 'sci-fi', 'war', 'sport', 'music', 'documentary', 'musical', 'western', 'short', 'film-noir', 'nan', 'talk-show', 'news', 'adult', 'reality-tv', 'game-show'])


In [530]:
#dependent vbs
music_cols=cols[:19]["short"]
movie_cols=cols[19:31]["short"]
hobbies=cols[31:63]["short"]

#independent vbs
personality=cols[76:133]["short"]
demographics=cols.iloc[[140, 144, 146]]["short"]

In [531]:
def gender_numeric(gender):   
    return {
        'female': 1,
        'male':0, 
    }.get(gender,float("nan"))

resp["Gender"]=list(map(gender_numeric, list(resp["Gender"])))
resp

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,1.0,right handed,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,1.0,right handed,college/bachelor degree,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,20.0,176.0,67.0,2.0,1.0,right handed,secondary school,no,city,block of flats
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,...,22.0,172.0,59.0,1.0,1.0,right handed,college/bachelor degree,yes,city,house/bungalow
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,20.0,170.0,59.0,1.0,1.0,right handed,secondary school,no,village,house/bungalow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,5.0,2.0,5.0,2.0,2.0,5.0,4.0,4.0,4.0,3.0,...,20.0,164.0,57.0,1.0,1.0,right handed,secondary school,no,city,house/bungalow
1006,4.0,4.0,5.0,1.0,3.0,4.0,1.0,4.0,1.0,1.0,...,27.0,183.0,80.0,5.0,0.0,left handed,masters degree,no,village,house/bungalow
1007,4.0,3.0,1.0,1.0,2.0,2.0,2.0,3.0,4.0,1.0,...,18.0,173.0,75.0,0.0,1.0,right handed,secondary school,yes,city,block of flats
1008,5.0,3.0,3.0,3.0,1.0,3.0,1.0,3.0,4.0,1.0,...,25.0,173.0,58.0,1.0,1.0,right handed,college/bachelor degree,no,city,block of flats


In [302]:
# Convert education to quantitative values and reassign to table
def edu_level_numeric(edu):   
    return {
        'currently a primary school pupil': 0,
        'primary school': 1,
        'secondary school': 2, 
        'college/bachelor degree': 3,
        'masters degree': 4,   
        'doctorate degree': 5
    }.get(edu,float("nan"))

resp["Education"]=list(map(edu_level_numeric, list(resp["Education"])))

In [503]:
music_resp=resp.iloc[:, :19]
movie_resp=resp.iloc[:, 19:31]
hobbies_resp=resp.iloc[:, 31:63]
personality_resp=resp.iloc[:, [79, 80, 105, 106, 109, 110, 113, 129, 132]]
demographics_resp=resp.iloc[:, [140, 144, 146, 147]]

final_mat=pd.concat([music_resp, movie_resp, hobbies_resp, personality_resp, demographics_resp], axis=1)
final_mat=final_mat.rename(columns={"New environment": "adapt", "Socializing": "meeting-people", "Waiting":"patient", "Number of friends": "friends", "Workaholism": "study", "Thinking ahead": "perspectives", "Charity": "charity", "Interests or hobbies": "differ-hobbies"})

corr_mat=pd.DataFrame.corr(final_mat)
corr_mat.columns = map(str.lower, corr_mat.columns)
corr_mat.index = map(str.lower, corr_mat.index)
corr_mat

,music,slow songs or fast songs,dance,folk,country,classical music,musical,pop,rock,metal or hardrock,...,perspectives,charity,friends,patient,adapt,meeting-people,differ-hobbies,age,gender,education
music,1.000000,0.100207,0.081572,0.031212,-0.007589,0.059276,0.064674,0.079583,0.165995,0.025163,...,0.005265,-0.029594,0.104106,0.024672,0.029275,0.083740,0.097759,-0.076829,0.102452,NaN
slow songs or fast songs,0.100207,1.000000,0.196041,-0.060910,-0.070274,-0.156435,-0.069897,0.064220,-0.006619,0.053267,...,-0.045324,-0.019243,0.082450,-0.016267,0.081480,0.099527,0.100607,0.012950,-0.078712,NaN
dance,0.081572,0.196041,1.000000,0.065654,0.058415,-0.089148,0.074097,0.431962,-0.134825,-0.234597,...,-0.016572,0.040269,0.163821,-0.013425,0.060805,0.161721,0.060151,0.031883,0.042760,NaN
folk,0.031212,-0.060910,0.065654,1.000000,0.386646,0.371015,0.263020,0.023270,0.059448,0.069513,...,0.067085,0.135071,0.068768,0.022994,0.025177,0.039159,0.125313,0.133230,0.040142,NaN
country,-0.007589,-0.070274,0.058415,0.386646,1.000000,0.256018,0.205943,0.008051,0.130418,0.112666,...,0.042029,0.124429,0.045707,0.087221,0.000647,-0.016398,0.069271,0.073303,-0.077075,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
meeting-people,0.083740,0.099527,0.161721,0.039159,-0.016398,-0.071689,0.000962,0.052097,-0.058155,-0.061008,...,-0.012132,0.064290,0.329125,0.072860,0.409969,1.000000,0.293300,-0.021718,0.006333,NaN
differ-hobbies,0.097759,0.100607,0.060151,0.125313,0.069271,0.072387,0.022032,-0.003239,-0.014361,0.053141,...,0.020508,0.105454,0.375824,0.078200,0.251919,0.293300,1.000000,0.043938,-0.092088,NaN
age,-0.076829,0.012950,0.031883,0.133230,0.073303,0.036074,0.015620,0.020049,0.000290,0.004631,...,0.074108,0.120679,0.025248,0.063409,-0.033647,-0.021718,0.043938,1.000000,-0.132559,NaN
gender,0.102452,-0.078712,0.042760,0.040142,-0.077075,-0.002501,0.289300,0.161292,-0.010918,-0.171514,...,-0.043002,0.044912,-0.012606,-0.051015,-0.067940,0.006333,-0.092088,-0.132559,1.000000,NaN


In [553]:
def recs(genre_dict, genre_query, corr_query):
    """Returns a list of podcasts based on interests user clicked in form
    Params: {query: list of genre names, genre_dict: dictionary that maps genre to titles}
    Returns: list of titles and scores
    """
    
    #query is an array of genre_names
    #corr_query is a dictionary of radio button responses like {'all-music': 'R&B', 'all-movies': 'Horror', 'all-interests': 'Social Sciences', 'gender': 'female', 'education': 'no'}
    if 'all_interests' in corr_query: 
        corr_query.pop('all-interests')
    if 'all_movies' in corr_query:
        corr_query.pop('all-movies')
    if 'all_music' in corr_query:
        corr_query.pop('all-music')
    genre_query=[genre.lower() for genre in genre_query]
    
    counter={}
    for key in corr_query:
        val=float(corr_query[key]) 
        for cat in genre_query: 
            corr=float(corr_mat[key][cat])
            weighted=corr*val
            for film in genre_dict[cat]:
                if film in counter:
                    counter[film]+=weighted
                else: 
                    counter[film]=weighted

    arr = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    subset = arr[:15]
    random.shuffle(subset)
    return subset[:5]

In [554]:
recs(genre_to_movie, ['Horror', 'Romance'], {'study': '1', 'perspectives': '1', 'charity': '1', 'friends': '5', 'patient': '5', 'adapt': '5', 'meeting-people': '5', 'differ-hobbies': '5'})

[('Pride and Prejudice', 1.6169716034500854),
 ('Silk (2007)', 1.6169716034500854),
 ('I Want Someone to Eat Cheese With (2006)', 1.6169716034500854),
 ('For the Bible Tells Me So (2007)', 1.6169716034500854),
 ('Mah nakorn (2004)', 1.6169716034500854)]

In [436]:
# Convert education to quantitative values and reassign to table
def edu_level_numeric(edu):   
    return {
        'currently a primary school pupil': 0,
        'primary school': 1,
        'secondary school': 2, 
        'college/bachelor degree': 3,
        'masters degree': 4,   
        'doctorate degree': 5
    }.get(edu,float("nan"))
resp["Education"]=list(map(edu_level_numeric, list(resp["Education"])))

In [47]:
def gender_numeric(gender):   
    return {
        'female': 0,
        'male':1
    }.get(gender,float("nan"))
resp["Gender"]=list(map(gender_numeric, list(resp["Gender"])))

In [48]:
# NOW FOR PERSONALITY
def punct_numeric(punct):   
    return {
        'i am often early': 1,
        'i am always on time':2,
        'i am often running late': 3
    }.get(punct,float("nan"))
resp["Punctuality"]=list(map(punct_numeric, list(resp["Punctuality"])))



def ly_numeric(lying):
    return {
        'never':0,
        'only to avoid hurting someone': 1,
        'sometimes': 2,
        'everytime it suits me': 3
    }.get(lying,float("nan"))
resp["Lying"]=list(map(ly_numeric,list(resp["Lying"])))


def web_numeric(web):
    return{
        'no time at all': 0, 
        'less than an hour a day': 1,
        'few hours a day': 2,
         'most of the day': 3
    }.get(web,float("nan"))
resp["Internet usage"]=list(map(web_numeric,list(resp["Internet usage"])))

In [49]:
corr_dict={}
for dem in demographics:
    dem_var=resp[dem]
    for col in music:
        music_var=resp[col]
        corr_dict[(col,dem)]=pd.concat([music_var,dem_var], axis=1).corr("pearson").iloc[0,1]
print("Correlations for Music and Demographic were between" , 
      sorted(corr_dict.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict.items(), key=lambda x: x[1])[-1])

n_corr_dict={}
for per in personality:
    per_var=resp[per]
    for col in music:
        music_var=resp[col]
        n_corr_dict[(col,per)]=pd.concat([music_var,per_var], axis=1).corr("pearson").iloc[0,1]

print("Correlations for Music and Personality were between" , 
      sorted(n_corr_dict.items(), key=lambda x: x[1])[0], "and",
      sorted(n_corr_dict.items(), key=lambda x: x[1])[-1])

Correlations for Music and Demographic were between (('Musical', 'Gender'), -0.2892995696964911) and (('Metal or Hardrock', 'Gender'), 0.17151380882446188)
Correlations for Music and Personality were between (('Latino', 'Criminal damage'), -0.1468954074190802) and (('Folk', 'God'), 0.22763210266420825)


In [50]:
# Movies
corr_dict1={}
for dem in demographics:
    dem_var=resp[dem]
    for col in movie_cols:
        movie_var=resp[col]
        corr_dict1[(col,dem)]=pd.concat([movie_var,dem_var], axis=1).corr("pearson").iloc[0,1]

print("Correlations for Movies and Demographics were between" , 
      sorted(corr_dict1.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict1.items(), key=lambda x: x[1])[-1])




corr_dict2={}
for per in personality:
    per_var=resp[per]
    for col in movie_cols:
        movie_var=resp[col]
        corr_dict2[(col,per)]=pd.concat([movie_var,per_var], axis=1).corr("pearson").iloc[0,1]

print("Correlations for Movies and Personality were between" , 
      sorted(corr_dict2.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict2.items(), key=lambda x: x[1])[-1])

Correlations for Movies and Demographics were between (('Romantic', 'Gender'), -0.4392148795776444) and (('War', 'Gender'), 0.4131725852187737)
Correlations for Movies and Personality were between (('Action', 'Life struggles'), -0.30515880791393) and (('Romantic', 'Life struggles'), 0.42042908699149656)


In [51]:
# Hobbies
corr_dict3={}
for dem in demographics:
    dem_var=resp[dem]
    for col in hobbies:
        hob=resp[col]
        corr_dict3[(col,dem)]=pd.concat([hob,dem_var], axis=1).corr("pearson").iloc[0,1]
        

print("Correlations for Hobbies and Demographics were between" , 
      sorted(corr_dict3.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict3.items(), key=lambda x: x[1])[-1])



corr_dict4={}
for per in personality:
    per_var=resp[per]
    for col in hobbies:
        hob=resp[col]
        corr_dict4[(col,per)]=pd.concat([hob,per_var], axis=1).corr("pearson").iloc[0,1]
        
print("Correlations for Hobbies and Personality were between" , 
      sorted(corr_dict4.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict4.items(), key=lambda x: x[1])[-1])


Correlations for Hobbies and Demographics were between (('Shopping', 'Gender'), -0.3872709887868457) and (('PC', 'Gender'), 0.4630847060120584)
Correlations for Hobbies and Personality were between (('Cars', 'Life struggles'), -0.3093165058617513) and (('Religion', 'God'), 0.50844462381186)


Index(['Music', 'Slow songs or fast songs', 'Dance', 'Folk', 'Country',
       'Classical music', 'Musical', 'Pop', 'Rock', 'Metal or Hardrock',
       ...
       'Age', 'Height', 'Weight', 'Number of siblings', 'Gender',
       'Left - right handed', 'Education', 'Only child', 'Village - town',
       'House - block of flats'],
      dtype='object', length=150)


,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,0.0,right handed,3.0,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,0.0,right handed,3.0,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,20.0,176.0,67.0,2.0,0.0,right handed,2.0,no,city,block of flats
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,...,22.0,172.0,59.0,1.0,0.0,right handed,3.0,yes,city,house/bungalow
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,20.0,170.0,59.0,1.0,0.0,right handed,2.0,no,village,house/bungalow
